In [2]:
pip install surprise

     |████████████████████████████████| 11.8 MB 10.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619396 sha256=d2b568e98fd1707a4332497541002cd7ff0d906ffb0f14638b102b70876bb177
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [87]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, normalize

from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
DATASET_LINK='http://files.grouplens.org/datasets/movielens/ml-100k.zip'
!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip
overall_stats = pd.read_csv('ml-100k/u.info', header=None)
print("Details of users, items and ratings involved in the loaded movielens dataset: ",list(overall_stats[0]))

--2021-10-10 14:42:15--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.70MB/s    in 0.8s    

2021-10-10 14:42:16 (5.70 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [7]:
## same item id is same as movie id, item id column is renamed as movie id
column_names1 = ['userId','movieId','rating','timestamp']
ratings_df = pd.read_csv('ml-100k/u.data', sep='\t',header=None,names=column_names1)
ratings_df.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [44]:
item_df = pd.read_csv('ml-100k/u.item', sep='|',header=None,encoding='latin-1')
item_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [45]:
movies_df = item_df[[0,1]].set_axis(['movieId', 'title'], axis=1)

In [46]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [26]:
# df = pd.merge(ratings_df, movies, how='inner', on='movieId')
# df.head()

In [27]:
# df.info()

In [28]:
# df['rating'].value_counts(sort=True).plot(kind='bar' ,figsize=(10,6), use_index = True, rot=0)
# plt.title('Bar plot of rating frequency')
# plt.xlabel('Rating')
# plt.ylabel('Number of times a rating was given')

In [29]:
# new_df=df.groupby("movieId").filter(lambda x:x['rating'].count() >=10000)
# new_df

In [30]:
# user_filter=df.groupby("userId").filter(lambda x:x['rating'].count() >=1000)
# user_filter

In [34]:
ratings_matrix = ratings_df.pivot_table(values='rating', index='movieId', columns='userId', fill_value=0)
ratings_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5,4,0,0,4,4,0,0,0,4,0,0,3,0,1,5,4,5,0,3,5,0,5,0,5,3,0,0,0,0,0,0,0,0,0,0,0,5,0,0,...,0,0,0,5,0,0,4,0,0,2,0,0,4,3,3,4,0,3,5,3,5,0,0,5,0,3,3,0,4,3,2,3,4,0,4,0,0,5,0,0
2,3,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,5
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,3,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
4,3,0,0,0,0,0,5,0,0,4,0,5,5,0,0,5,0,3,4,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,0,0,0,0,2,0,0,0
5,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0,0,0,3,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
ratings_matrix.shape

(1682, 943)

# top 5 movies based on a given movie title

## Using NearestNeighbors

In [40]:
from scipy.sparse import csr_matrix
csr_data = csr_matrix(ratings_matrix.values)
# movies_ratings_matrix.reset_index(inplace=True)
print(csr_data[:5,:5])

  (0, 0)	5
  (0, 1)	4
  (0, 4)	4
  (1, 0)	3
  (1, 4)	3
  (2, 0)	4
  (3, 0)	3
  (4, 0)	3


In [41]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [83]:
def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 5
    movie_list = movies_df[movies_df['title'].str.contains(movie_name)]  
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId']
        reset = ratings_matrix.reset_index()
        movie_idx = reset[reset['movieId'] == movie_idx].index[0]
        distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = reset.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    else:
        return "No movies found. Please check your input"

In [84]:
get_movie_recommendation('Toy Story')

,Title,Distance
1,Mission: Impossible (1996),0.358678
2,"Rock, The (1996)",0.335445
3,Independence Day (ID4) (1996),0.310214
4,Return of the Jedi (1983),0.300075
5,Star Wars (1977),0.265428


## NMF

In [88]:
nmf=NMF(n_components=10)
movies_W=nmf.fit_transform(ratings_matrix)
movies_H=nmf.components_

print('w=',movies_W)
print('#'*60)
print('H=',movies_H)
print(movies_W.shape,movies_H.shape)

#normalize the value between 0,1 as prcentages
movies_w = normalize(movies_W)

w= [[5.60994285e-01 2.60836765e+00 1.01709972e-01 ... 0.00000000e+00
  0.00000000e+00 2.81407405e-01]
 [6.36779953e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.62860476e+00 1.31804908e-01]
 [0.00000000e+00 4.62709489e-01 0.00000000e+00 ... 0.00000000e+00
  2.23300747e-01 9.00249156e-01]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.70764453e-03 0.00000000e+00 0.00000000e+00 ... 2.82844499e-04
  1.61898365e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.01927521e-02 ... 0.00000000e+00
  2.44643221e-03 2.41833240e-02]]
############################################################
H= [[0.65354653 0.         0.03426351 ... 0.         0.50309328 0.95740807]
 [0.05771417 0.70861592 0.         ... 0.48073512 0.         0.3364942 ]
 [1.24706831 0.42660949 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.57092347 0.        ]
 [0.72335221 0.         0.         .

## Using Dot Product

In [89]:
df3=pd.DataFrame(movies_w,index=ratings_matrix.index)
df3.head()

,0,1,2,3,4,5,6,7,8,9
movieId,,,,,,,,,,
1,0.149221,0.693811,0.027054,0.000000,0.000000,0.187712,0.674381,0.000000,0.000000,0.074853
2,0.003764,0.000000,0.000000,0.000000,0.000000,0.183237,0.182943,0.000000,0.962741,0.077916
3,0.000000,0.444584,0.000000,0.000000,0.000000,0.090076,0.000000,0.000000,0.214554,0.864984
4,0.371477,0.000000,0.237786,0.000000,0.000000,0.330765,0.024324,0.094766,0.620290,0.549296
5,0.000000,0.127300,0.000000,0.105451,0.000000,0.525395,0.000000,0.000000,0.560135,0.618776
...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.288448,0.957495,0.000000,0.000000,0.000000,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.288448,0.957495,0.000000,0.000000,0.000000,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.288448,0.957495,0.000000,0.000000,0.000000,0.000000,0.000000


In [127]:
def get_movie_recommendation_dot(movie_name):
    movie_list = movies_df[movies_df['title'].str.contains(movie_name)]  
    if len(movie_list):        
        movie_idx= movie_list.iloc[0]['movieId']
        print(movie_idx)
        user = df3.loc[movie_idx]
        dot = df3.dot(user)
        output = dot.nlargest()
        
        recommend_frame = []
        for val in output.index:
          print(val)
            # idx = movies[movies['movieId'] == movie_idx].index
          recommend_frame.append({'Title':movies.iloc[val]['title'],'percentage':output[val]})
        df = pd.DataFrame(recommend_frame,index=range(1,len(output)+1))
        return df
    else:
        return "No movies found. Please check your input"

In [128]:
get_movie_recommendation_dot('Toy Story')

1
1
1060
181
257
151


,Title,percentage
1,GoldenEye (1995),1.000000
2,"Evening Star, The (1996)",0.954845
3,GoodFellas (1990),0.949965
4,Contact (1997),0.948164
5,Sleeper (1973),0.938726


In [112]:
#instead of reader in surprise

user = df3.loc[1]
output = df3.dot(user)
print(output.nlargest())


movieId
1       1.000000
1060    0.954845
181     0.949965
257     0.948164
151     0.938726
dtype: float64


1.0

In [122]:
movies.iloc[1]

movieId                   2
title      GoldenEye (1995)
Name: 1, dtype: object

## NMF

In [ ]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, normalize

meow=NMF(n_components=10)
W=meow.fit_transform(ratings_matrix)
H=meow.components_

print('w=',W)
print('#'*60)
print('H=',H)
print(W.shape,H.shape)

#normalize the value between 0,1 as prcentages
new_w = normalize(W)

C:\Users\Ahmed Ayman\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


w= [[0.25571032 1.25702605 0.         ... 0.         0.24729582 0.        ]
 [0.50212478 0.         0.00626453 ... 0.97729055 0.36902758 0.        ]
 [0.29030188 1.15168256 0.88144954 ... 0.60385225 0.         0.        ]
 ...
 [0.85095149 0.40340523 0.         ... 0.         0.         0.        ]
 [0.45311943 0.03534181 0.23599574 ... 0.23592702 0.41330337 0.33707591]
 [0.         0.87731927 0.46686951 ... 0.79084782 0.38870222 0.71661128]]
############################################################
H= [[1.52469656e+00 1.28127802e+00 6.14252337e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.69932392e+00 1.45795667e+00 0.00000000e+00 ... 1.20447737e-04
  9.00173672e-05 0.00000000e+00]
 [8.54470302e-01 2.82038309e-01 0.00000000e+00 ... 5.26096965e-03
  1.02266831e-03 3.95612304e-03]
 ...
 [7.02376097e-01 1.40286764e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.11999176e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.000

In [ ]:
df2=pd.DataFrame(new_w,index=ratings_matrix.index)
df2

,0,1,2,3,4,5,6,7,8,9
userId,,,,,,,,,,
187,0.120741,0.593543,0.000000,0.000000,0.107160,0.779457,0.021390,0.000000,0.116768,0.000000
426,0.409558,0.000000,0.005110,0.315007,0.083651,0.000000,0.000000,0.797126,0.300997,0.000000
541,0.181660,0.720678,0.551577,0.024531,0.000000,0.000000,0.000000,0.377867,0.000000,0.000000
548,0.088972,0.000000,0.000000,0.000000,0.061437,0.979048,0.147022,0.000000,0.090328,0.000000
626,0.149666,0.114073,0.000000,0.736685,0.162119,0.000000,0.627495,0.000000,0.000000,0.043000
...,...,...,...,...,...,...,...,...,...,...
162047,0.197851,0.000000,0.640348,0.000000,0.000000,0.416230,0.110470,0.267018,0.044727,0.540425
162271,0.000000,0.006518,0.450750,0.489527,0.023101,0.000000,0.146617,0.635096,0.228563,0.282006
162495,0.694518,0.329246,0.000000,0.000000,0.205723,0.252073,0.550798,0.000000,0.000000,0.000000


## Using Dot Product

In [ ]:
#instead of reader in surprise

user = df2.loc[426]
output = df2.dot(user)
print(output.nlargest())

userId
426       1.000000
16552     0.972939
112307    0.953833
99843     0.953270
62008     0.951363
dtype: float64


In [ ]:
print(output)

userId
187       0.093561
426       1.000000
541       0.386154
548       0.068767
626       0.306920
            ...   
162047    0.310613
162271    0.733489
162495    0.301654
162508    0.699888
162516    0.460511
Length: 2675, dtype: float64


In [ ]:
movies_ratings_matrix = ratings_matrix.T
movies_ratings_matrix.head()

userId,187,426,541,548,626,653,757,803,846,847,...,161560,161586,161675,161826,161928,162047,162271,162495,162508,162516
movieId,,,,,,,,,,,,,,,,,,,,,
1,3.5,2.5,5.0,4.5,4.5,5.0,3.0,5.0,0.0,4.0,...,4.0,3.0,3.5,3.0,3.0,3.0,4.0,3.0,4.5,4.5
2,3.5,0.0,4.5,4.0,4.0,3.5,3.0,0.0,0.0,0.0,...,3.5,1.5,2.5,1.0,4.0,2.5,1.0,3.0,0.0,2.5
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.5,0.0,3.5,0.0,0.5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
